In [1]:
import os
import gc
import pandas as pd
import numpy as np
import pickle
import pytorch_lightning as pl
from src import CustomGraphMultipleChoiceEncoder, GraphProcessorV2
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch, Data
from scipy.stats import rankdata
from tqdm.auto import tqdm, trange
from sklearn.metrics import accuracy_score
from transformers import get_cosine_schedule_with_warmup,get_linear_schedule_with_warmup, AutoModel, AutoTokenizer, AutoModelForMultipleChoice, AutoConfig
import wandb
import random
pl.seed_everything(56)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Seed set to 56


In [1]:
#bdmp = pickle.load(open('csqa_data/maper_inv.pkl','rb'))
#new_bdmp = {}
#for r in dm.graph_state['concepts_maper']:
#    i =  dm.graph_state['concepts_maper'][r]
#    r_s = bdmp[i]
#    if type(r_s) == float:
#        continue
#    else:
#        r_s = r_s.replace(' ','_')
#    new_bdmp[r] = dm.graph_state['concepts_maper'][r_s]

In [2]:
class CFG:
    class data:
        train_path = 'csqa_data/train.statement.jsonl'
        dev_path = 'csqa_data/dev.statement.jsonl'
        test_path = 'csqa_data/test.statement.jsonl'
        dream_train_path = 'data/train.statement.jsonl'
        dream_dev_path = 'data/dev.statement.jsonl'
        dream_test_path = 'data/test.statement.jsonl'
        dream_train_concept_path = 'prep_data/train_choices.csv'
        dream_dev_concept_path = 'prep_data/vall_choices.csv'
        dream_test_concept_path = 'prep_data/test_choices.csv'
        tokenizer = "microsoft/deberta-v3-large"
        graph_path = 'graph.pkl'
        embeds_path = 'deberta_large_embedsV2.npy'
        maper_path = 'csqa_data/maper_inv.pkl'
        concepts_lst = 'maper_v2.pkl'
        #concepts_lst = 'csqa_data/maper.pkl'
        dop_concepts_csqa_path = 'csqa_data/all_candsV3.pkl'
        dop_concepts_dream_path = 'dream_all_candidates.pkl'
        num_workers = 0
        pad_to_nodes = 64
        nfolds = 5
        batch_size = 4
        use_prefix = True
        max_length = 384 
        seed = 56
    class model:
        model = "microsoft/deberta-v3-large"
        optim = torch.optim.AdamW
        use_only_encoder = False
        use_tkds = True
        grad_acum_steps = 1
        not_padding_token = False
        torch_dtype = None
        scheduler = 'cosine'
        warmup_steps = 0.0 #0.25
        num_labels = 1
        label_smoothing = 0.0
        lr = 1e-5
        lr_gnn = 1e-5
        lr_fn = 1e-4
        cls_drop_type = None
        cls_drop = 0.0
        pool = 'attention'
        max_epoches = 5
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        turn_off_drop = True
        num_cycles = 0.5
        eps = 1e-7
        weight_decay = 0.0
        weight_decay_fn = 0.0
        betas = (0.9, 0.999)
        use_lora = False
        class gnn:
            pool_type = 'no'
            num_nodes = 64
            max_nodes = 64
            edge_dim = 17
            blocks = [
                {'block_type':'gatv2','act':nn.GELU(),'block_norm':None,'drop_p':0.0,'input_dim':1024,'out_dim':1024,'heads':2},
                {'block_type':'gatv2','act':nn.GELU(),'block_norm':None,'drop_p':0.0,'input_dim':2048,'out_dim':1024,'heads':1},
                {'block_type':'mlp','act':nn.Identity(),'block_norm':None,'drop_p':0.0,'input_dim':1024,'out_dim':1024},
            ]

    seed = 56
    fold_number = 0

def set_wandb_cfg():
    config = {}
    for k,v in CFG.model.__dict__.items():
        if '__' not in k:
            config[k] = v
    for k,v in CFG.data.__dict__.items():
        if '__' not in k:
            config[k] = v
    config['fold_number'] = CFG.fold_number
    return config

NameError: name 'torch' is not defined

In [4]:
def make_df_csqa(path,is_test=False):
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question_state'] = data['question'].apply(lambda x: x['question_concept'])
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = 0
    return df

In [5]:
def make_df_dream(path,concept_path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': row[0],
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df).join(pd.read_csv(concept_path)).drop(columns=['choice_0', 'choice_1', 'choice_2']).rename(columns={'choice_1_concept': 'choice_0', 'choice_2_concept': 'choice_1', 'choice_3_concept': 'choice_2'})

In [6]:
df = make_df_csqa(CFG.data.train_path)

In [7]:
df

question_state                                           question  \
0         punishing  The sanctions against the school were a punish...   
1            people  Sammy wanted to go to where the people were. W...   
2            choker  To locate a choker not located in a jewelry bo...   
3           highway  Google Maps and other highway and street GPS s...   
4             cable  What home entertainment equipment requires cable?   
...             ...                                                ...   
7888            cat                     What are cats often known for?   
7889        landing  As he looked out the window, he knew the landi...   
7890          grape  If you really wanted a grape, where would you ...   
7891          climb  What would you do to a rock when climb up a cl...   
7892           hose  His compressor needed a new hose, where did he...   

                choice_A         choice_B       choice_C     choice_D  \
0                 ignore          enforce  authoritarian      yell at   
1             race track  populated areas     the desert    apartment   
2          jewelry store             neck    jewlery box  jewelry box   
3          united states           mexico    countryside        atlas   
4            radio shack       substation        cabinet   television   
...                  ...              ...            ...          ...   
7888            whiskers      sharp teeth           purr    four legs   
7889  apartment building        disembark      stairwell      deplane   
7890              winery      fruit stand          field      kitchen   
7891                grab        look down          throw      falling   
7892         garden shed   hardware store        brothel       garage   

         choice_E  label  
0           avoid      0  
1       roadblock      1  
2        boutique      0  
3          oceans      3  
4            desk      3  
...           ...    ...  
7888  sharp claws      4  
7889      airport      4  
7890         food      1  
7891     may fall      0  
7892   greenhouse      1  

[7893 rows x 8 columns]

In [8]:
class CsqaDataset(Dataset):
    def __init__(self, df, tokenizer, graph_miner, concepts):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.tokenizer = tokenizer
        self.graph_miner = graph_miner
        self.concepts = concepts
        
    def __len__(self):
        return len(self.data)

    def prepare_prompt(self, question, concepts, answer):
        s = f"Question: {question} "
        return s, f"Answer: {answer}"
    
    def parse_answer(self, answer, padd_node=160892):
        answer = answer.replace(' ','_')[:]
        if answer in self.concepts:
            res = self.concepts[answer]
            if res == -100:
                return padd_node
            return res
        else:
            return padd_node
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        
        if self.cfg.use_prefix:
            prompts = [
                self.prepare_prompt(row['question'], row['concepts'], i)
                for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]
            ]
        else:
            prompts = [
                [row['question'],i] # old ver
                for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]
            ]
        
        encodes = [
            self.tokenizer.encode_plus(
                question,answer,
                max_length=self.cfg.max_length,
                truncation='only_first',
                padding='max_length',
                return_tensors='pt'
            )
            for question,answer in prompts
        ]
        answers = [x for x in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]]
        graph_encs = [
            Data(**self.graph_miner.process([self.parse_answer(x) for x in ([a]+row['concepts'])]))
            for a in answers
        ]
        return {
            'input_ids':torch.stack([x.input_ids.squeeze(0) for x in encodes]),
            'attention_mask':torch.stack([x.attention_mask.squeeze(0) for x in encodes]),
            'token_type_ids':torch.stack([x.token_type_ids.squeeze(0) for x in encodes]),
            'graph_encodes': graph_encs,
            #'answers':answers,
            'labels': row['label']
        }

In [9]:
class DreamDataset(Dataset):
    def __init__(self, df, tokenizer, graph_miner, concepts):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.tokenizer = tokenizer
        self.graph_miner = graph_miner
        self.concepts = concepts
        
    def __len__(self):
        return len(self.data)

    def prepare_prompt(self, dialog, question, concepts, answer):
        s = f"Dialog: {dialog}[SEP]Question: {question} "
        return s, f"Answer: {answer}"
    
    def parse_answer(self, answer, padd_node=160892):
        answer = answer.replace(' ','_')[:]
        if answer in self.concepts:
            res = int(self.concepts[answer])
            if res == -100:
                return padd_node
            return res
        else:
            return padd_node

    def parse_dialog(self, dialog):
        return '[SEP]'.join(dialog)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        dialog = self.parse_dialog(row['dialog'])
        if self.cfg.use_prefix:
            prompts = [
                self.prepare_prompt(dialog, row['question'], row['concepts'], i)
                for i in [row['choice_0'],row['choice_1'],row['choice_2']]
            ]
        else:
            prompts = [
                [row['question'],i] # old ver
                for i in [row['choice_0'],row['choice_1'],row['choice_2']]
            ]
        
        encodes = [
            self.tokenizer.encode_plus(
                question,answer,
                max_length=self.cfg.max_length,
                truncation='only_first',
                padding='max_length',
                return_tensors='pt'
            )
            for question,answer in prompts
        ]
        answers = [x for x in [row['choice_0'],row['choice_1'],row['choice_2']]]
        graph_encs = [
            Data(**self.graph_miner.process([self.parse_answer(x) for x in ([a]+row['concepts'])]))
            for a in answers
        ]
        return {
            'input_ids':torch.stack([x.input_ids.squeeze(0) for x in encodes]),
            'attention_mask':torch.stack([x.attention_mask.squeeze(0) for x in encodes]),
            'token_type_ids':torch.stack([x.token_type_ids.squeeze(0) for x in encodes]),
            'graph_encodes': graph_encs,
            #'answers':answers,
            'labels': row['labels']
        }

In [10]:
from typing import List, Optional
import time


class IterLoader:
    def __init__(self, dataloader: DataLoader, use_distributed: bool = False):
        self._dataloader = dataloader
        self.iter_loader = iter(self._dataloader)
        self._use_distributed = use_distributed
        self._epoch = 0

    @property
    def epoch(self) -> int:
        return self._epoch

    def __next__(self):
        try:
            data = next(self.iter_loader)
        except StopIteration:
            self._epoch += 1
            if hasattr(self._dataloader.sampler, "set_epoch") and self._use_distributed:
                self._dataloader.sampler.set_epoch(self._epoch)
            time.sleep(0.1)  # Prevent possible deadlock during epoch transition
            self.iter_loader = iter(self._dataloader)
            data = next(self.iter_loader)

        return data

    def __iter__(self):
        return self

    def __len__(self):
        return len(self._dataloader)


class MultiIterLoader:
    def __init__(
        self, loaders: List[DataLoader], ratios: Optional[List[float]] = None, names: List[str] = None, sequential=True
    ):
        self.min_loader_size = 0
        self.max_loader_size = 0
        for loader in loaders:
            assert hasattr(loader, "__next__"), "Loader {} has no __next__ method.".format(loader)
            if hasattr(loader, "__len__"):
                length = len(loader)
                if self.min_loader_size == 0:
                    self.min_loader_size = length
                elif length <= self.min_loader_size:
                    self.min_loader_size = length
                elif length >= self.max_loader_size:
                    self.max_loader_size = length

        if ratios is None:
            ratios = [1.0] * len(loaders)
        else:
            assert len(ratios) == len(loaders)
            ratios = [float(ratio) / sum(ratios) for ratio in ratios]

        self.loaders = loaders
        self.ratios = ratios
        self.sequential = sequential
        self.loader_idx = 0
        if names:
            self.names = names

    def __next__(self):
        if self.sequential:
            loader_idx = self.loader_idx
        else:
            # random sample from each loader by ratio
            loader_idx = random.choices(range(len(self.loaders)), self.ratios, k=1)[0]
        self.loader_idx = (loader_idx + 1) % len(self.loaders)
        if self.names:
            return next(self.loaders[loader_idx]), self.names[loader_idx]
        return next(self.loaders[loader_idx])

    def __iter__(self):
        return self

    def __len__(self):
        return self.min_loader_size

In [11]:
class AlternatingDataLoader:
    def __init__(self, loader1, loader2):
        self.loader1 = loader1
        self.loader2 = loader2
        self.iter1 = iter(loader1)
        self.iter2 = iter(loader2)
        self.exhausted1 = False
        self.exhausted2 = False

    def __iter__(self):
        return self

    def get_from_loader1(self):
        try:
            return next(self.iter1)
        except StopIteration:
            self.exhausted1 = True

    def get_from_loader2(self):
        try:
            return next(self.iter2)
        except StopIteration:
            self.exhausted2 = True

    def __next__(self):
        if not self.exhausted1:
            sample1 = self.get_from_loader1()
            if sample1 is not None:
                return sample1
            
        if not self.exhausted2:
            sample2 = self.get_from_loader2()
            if sample2 is not None:
                return sample2
        if self.exhausted1 and self.exhausted2:
            raise StopIteration

    def __len__(self):
        return len(self.loader1) + len(self.loader2)

In [12]:
class RandomDataLoader:
    def __init__(self, loader1, loader2):
        self.loader1 = loader1
        self.loader2 = loader2
        self.iter1 = iter(loader1)
        self.iter2 = iter(loader2)
        self.exhausted1 = False
        self.exhausted2 = False

    def __iter__(self):
        return self

    def get_from_loader1(self):
        try:
            return next(self.iter1)
        except StopIteration:
            self.exhausted1 = True

    def get_from_loader2(self):
        try:
            return next(self.iter2)
        except StopIteration:
            self.exhausted2 = True

    def __next__(self):
        if not self.exhausted1 and not self.exhausted2:
            sample = random.choice([self.get_from_loader1, self.get_from_loader2])()
            if sample is not None:
                return sample
        if not self.exhausted1:
            sample1 = self.get_from_loader1()
            if sample1 is not None:
                return sample1
            
        if not self.exhausted2:
            sample2 = self.get_from_loader2()
            if sample2 is not None:
                return sample2
        if self.exhausted1 and self.exhausted2:
            raise StopIteration

    def __len__(self):
        return len(self.loader1) + len(self.loader2)

In [13]:
def alternate_dataloaders(loader1, loader2):
    iter1, iter2 = iter(loader1), iter(loader2)
    exhausted1, exhausted2 = False, False

    while not (exhausted1 and exhausted2):
        if not exhausted1:
            try:
                yield next(iter1)
            except StopIteration:
                exhausted1 = True

        if not exhausted2:
            try:
                yield next(iter2)
            except StopIteration:
                exhausted2 = True

In [14]:
def sequence_dataloaders(loader1, loader2):
    iter1, iter2 = iter(loader1), iter(loader2)
    exhausted1, exhausted2 = False, False

    while not exhausted1:
        if not exhausted1:
            try:
                yield next(iter1)
            except StopIteration:
                exhausted1 = True
    while not exhausted2:
        if not exhausted2:
            try:
                yield next(iter2)
            except StopIteration:
                exhausted2 = True

In [15]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
        
    def prepare_data(self):
        if self.is_prepared: return None
        self.train_df_csqa = make_df_csqa(self.cfg.train_path)
        self.val_df_csqa = make_df_csqa(self.cfg.dev_path)
        self.test_df_csqa = make_df_csqa(self.cfg.test_path,is_test=True)

        self.train_df_dream = make_df_dream(self.cfg.dream_train_path, self.cfg.dream_train_concept_path)
        self.val_df_dream = make_df_dream(self.cfg.dream_dev_path, self.cfg.dream_dev_concept_path)
        self.test_df_dream = make_df_dream(self.cfg.dream_test_path, self.cfg.dream_test_concept_path,is_test=True)
        
        self.dop_concepts_csqa = pickle.load(open(self.cfg.dop_concepts_csqa_path, 'rb'))
        self.maper = pickle.load(open(self.cfg.maper_path, 'rb'))
        self.concepts = pickle.load(open(self.cfg.concepts_lst,'rb'))
        self.train_df_csqa['concepts'] = self.dop_concepts_csqa['train_all_cands']
        self.val_df_csqa['concepts'] = self.dop_concepts_csqa['vall_all_cands']
        self.test_df_csqa['concepts'] = self.dop_concepts_csqa['test_all_cands']

        self.dop_concepts_dream = pickle.load(open(self.cfg.dop_concepts_dream_path, 'rb'))
        print(self.dop_concepts_dream.keys())
        self.train_df_dream['concepts'] = self.dop_concepts_dream['train_candidates']
        self.val_df_dream['concepts'] = self.dop_concepts_dream['vall_candidates']
        self.test_df_dream['concepts'] = self.dop_concepts_dream['test_candidates']
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg.tokenizer, truncation_side='left')
        self.graph_state = pickle.load(open(self.cfg.graph_path, 'rb'))
        self.embeds = np.load(self.cfg.embeds_path)
        self.graph_miner = GraphProcessorV2(self.graph_state['graph'], self.embeds,pad_to_nodes=self.cfg.pad_to_nodes)
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup: return None
        #kf = MultilabelStratifiedKFold(n_splits=self.cfg.nfolds, shuffle=True, random_state=self.cfg.seed)
        #splits = [(x,y) for x,y in  kf.split(self.df.values,np.stack(self.df['label'].values))][CFG.fold_number]
        #self.train_df, self.val_df = self.df.iloc[splits[0]], self.df.iloc[splits[1]]
        self.train_dataset_csqa = CsqaDataset(self.train_df_csqa,self.tokenizer, self.graph_miner,self.concepts)
        self.val_dataset_csqa = CsqaDataset(self.val_df_csqa,self.tokenizer, self.graph_miner,self.concepts)
        self.predict_dataset_csqa = CsqaDataset(self.test_df_csqa,self.tokenizer, self.graph_miner,self.concepts)

        self.train_dataset_dream = DreamDataset(self.train_df_dream,self.tokenizer, self.graph_miner,self.concepts)
        self.val_dataset_dream = DreamDataset(self.val_df_dream,self.tokenizer, self.graph_miner,self.concepts)
        self.predict_dataset_dream = DreamDataset(self.test_df_dream,self.tokenizer, self.graph_miner,self.concepts)
        self.is_setup = True
    
    def train_dataloader(self):
        csqa_loader = DataLoader(self.train_dataset_csqa,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         collate_fn=collate_fn,
                         pin_memory=True,
                         shuffle=True)
        dream_loader = DataLoader(self.train_dataset_dream,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         collate_fn=collate_fn,
                         pin_memory=True,
                         shuffle=True)
        dataloader_all = RandomDataLoader(csqa_loader, dream_loader)
        return dataloader_all
    
    def val_dataloader(self):
        csqa_loader = DataLoader(self.val_dataset_csqa,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dream_loader = DataLoader(self.val_dataset_dream,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dataloader_all = AlternatingDataLoader(csqa_loader, dream_loader)
        return dataloader_all
    
    def predict_dataloader(self):
        csqa_loader = DataLoader(self.predict_dataset_csqa,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dream_loader = DataLoader(self.predict_dataset_dream,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dataloader_all = AlternatingDataLoader(csqa_loader, dream_loader)
        return dataloader_all

def compute_idxes(batch):
    idxes = []
    for i in range(len(batch)):
        if batch[i] != batch[i-1]:
            idxes.append(i)
    return torch.tensor(idxes)    
    
def collate_fn(batches):
    new_batch = {}
    for k in batches[0].keys():
        if k == 'graph_encodes':
            values = []
            for x in batches:
                values += x[k]
            
            try:
                new_batch[k] = Batch.from_data_list(values)
            except:
                print(values)
                new_batch[k] = Batch.from_data_list(values)
        elif k == 'labels':
            new_batch[k] = torch.from_numpy(np.array([x[k] for x in batches]))
        else:
            new_batch[k] = torch.stack([x[k] for x in batches])
            
    return new_batch

In [16]:
class AverageMeter():
    def __init__(self):
        self.preds = []
        self.labels = []
        self.history = []
    
    def update(self,y_t,y_p):
        self.labels += y_t
        self.preds += y_p
        
    def clean(self):
        self.preds = []
        self.labels = []

    def calc_metrics(self):
        metrics = {}
        metrics['accuracy'] = accuracy_score(self.labels, self.preds)
        
        self.history.append(metrics)
        return metrics

In [17]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.model = CustomGraphMultipleChoiceEncoder(self.cfg)
        self.avg_meter = AverageMeter()
        
    def forward(self, batch):
        output = self.model(**batch)
        return output

    def training_step(self, batch, i):
        out = self(batch)
        loss = out.loss
        self.log('train_loss', loss.item())
        return loss
            
    def validation_step(self, batch, i):
        out = self(batch)
        loss = out.loss
        self.log('val_loss',loss.item())
        preds = out.logits.argmax(dim=-1).tolist()
        self.avg_meter.update(batch['labels'].tolist(),preds)
    
    def predict_step(self, batch, i):
        out = self(batch)
        logits = out.logits
        return logits.argmax(dim=-1).tolist()
                
    def on_validation_epoch_end(self):
        metrics = self.avg_meter.calc_metrics()
        self.log_dict(metrics)
        self.avg_meter.clean()
            
    def configure_optimizers(self):        
        optimizer_parameters = [
            {'params': [p for n, p in self.model.model.named_parameters() if not any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': [p for n, p in self.model.model.named_parameters() if any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': 0.0},
            {'params': [p for n, p in self.model.gnn.named_parameters()],
             'lr': self.cfg.lr_gnn, 'weight_decay': self.cfg.weight_decay},
            {'params': [p for n, p in self.model.fc.named_parameters()],
             'lr': self.cfg.lr_fn, 'weight_decay': self.cfg.weight_decay_fn}
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_steps,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_steps)
        else:
            return optim
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [18]:
dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

dict_keys(['train_candidates', 'test_candidates', 'vall_candidates'])


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [21]:
with open('graph_miner.pkl','wb') as f:
    pickle.dump(dm.graph_miner,f)

In [19]:
CFG.model.num_training_steps = len(dm.train_dataloader()) * CFG.model.max_epoches
model = PLModule().cuda()

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [20]:
wandb.login(key="bf4dab65b16e4b3cc78348f00f79cba6db3d5f58")
os.environ['WANDB_API_KEY'] = "bf4dab65b16e4b3cc78348f00f79cba6db3d5f58"
wandb.init(project='AIIJC_stage_2',name='deberta-v3-large_inject_graph',config=set_wandb_cfg())

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: danasone. Use `wandb login --relogin` to force relogin


In [21]:
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
checkpoint_cb = pl.callbacks.ModelCheckpoint(
    dirpath='./outputs/',
    filename='model_{epoch:02d}-{accuracy:.4f}',
    monitor='accuracy',
    mode='max',
    save_last=True
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=32,
    callbacks = [lr_monitor],#[lr_monitor,checkpoint_cb],
    logger = pl.loggers.WandbLogger(save_code=True),
    log_every_n_steps=1,
    accumulate_grad_batches=CFG.model.grad_acum_steps,
    enable_checkpointing=False,
    min_epochs=1,
    devices=1,
    check_val_every_n_epoch=1,
    max_epochs=CFG.model.max_epoches,
    gradient_clip_val=1.0
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [22]:
trainer.fit(model, datamodule=dm)

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                             | Params | Mode 
-------------------------------------------------------------------
0 | model | CustomGraphMultipleChoiceEncoder | 444 M  | train
-------------------------------------------------------------------
444 M     Trainable params
0         Non-trainable params
444 M     Tot

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/notebooks/src/graph_proc.py:91: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

/notebooks/src/graph_proc.py:90: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  'edge_index': torch.tensor(edge_index).T,


RuntimeError: scatter(): Expected dtype int64 for index

In [21]:
torch.tensor(5.0).long()

tensor(5)